Student's Name and Email Address

Boise State University, Department of Chemistry and Biochemistry

## CHEM 324: PChem Lab
# Worksheet 2: Kinetics of Methylene-Blue RedOx

In [ ]:
# Import the main modules used in this worksheet
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
from sklearn.linear_model import LinearRegression
import scipy.optimize

In [ ]:
# Load the google drive with your files 
from google.colab import drive
drive.mount('/content/drive')
# The following needs to be the path of the folder with all your collected data in .csv format
path='/content/drive/MyDrive/Colab Notebooks/Kinetics_Data/'
path='Kinetics_Data/'

In [ ]:
# As a preliminary step, you need to create a .csv file with the important 
# information on each experiment and the corresponding filename.
# You can use the template provided by the instructors, but make sure check
# your experimental data and filenames
concentration_file='concentrations.csv'

In [ ]:
# The following commands will read all the .csv files and will create a 
# DataFrame with all the absorbance curves for the different experiments
#
concentrations=pd.read_csv(path+concentration_file)
maxtime = 0.
for index,row in concentrations.iterrows() : 
    data=pd.read_csv(path+row['Filename'],header=1,usecols=(0,1))
    data=data.apply(pd.to_numeric,errors='coerce').dropna()
    time = data.loc[len(data.index)-1,'Time (sec)']
    if time > maxtime : maxtime = time
full_data=pd.DataFrame({'time':np.arange(0.25,maxtime,0.5)})
for index,row in concentrations.iterrows() : 
    data=pd.read_csv(path+row['Filename'],header=1,usecols=(0,1))
    data=data.apply(pd.to_numeric,errors='coerce').dropna()
    full_data['abs'+row['Set']+str(row['Run'])]=data['Abs']

### Task 1: Reaction Order with Respect to Methylene-Blue

Based on your analyses, is the order with respect to methylene blue first or second? What evidence do you have for this finding?

## Task 2: EFfect of initial of [MB] on rate constant

In [ ]:
full_data

In [ ]:
plt.plot(full_data['time'],full_data['absA1'])
plt.plot(full_data['time'],full_data['absA2'])
plt.plot(full_data['time'],full_data['absA3'])
plt.plot(full_data['time'],full_data['absA4'])

In [ ]:
plt.semilogy(full_data['time'],full_data['absA1'])
plt.semilogy(full_data['time'],full_data['absA2'])
plt.semilogy(full_data['time'],full_data['absA3'])
plt.semilogy(full_data['time'],full_data['absA4'])

In [ ]:
def monoExp(x, m, t, b):
    return m * np.exp(-x/t) + b
def biExp(x, m1, t1, m2, t2, b):
    return m1 * np.exp(-x/t1) + m2 * np.exp(-x/t2) + b

In [ ]:
x,y=np.split(full_data[full_data['time']>0][['time','abs'+'A'+'1']].dropna().values,2,1)

In [ ]:
p0 = (1., 45., 0.05)
params, cv = scipy.optimize.curve_fit(monoExp, x.flatten(), y.flatten(), p0)
params

In [ ]:
p0 = (1., 45., 0.05)
for index,row in concentrations.iterrows() : 
    x,y=np.split(full_data[full_data['time']>80][['time','abs'+row['Set']+str(row['Run'])]].dropna().values,2,1)
    params, cv = scipy.optimize.curve_fit(monoExp, x.flatten(), y.flatten(), p0)
    concentrations.loc[index,'scale']=params[0]
    concentrations.loc[index,'scale_sigma']=np.sqrt(cv[0,0])
    concentrations.loc[index,'k']=1./params[1]
    concentrations.loc[index,'k_sigma']=1./params[1]**2*np.sqrt(cv[1,1])
    concentrations.loc[index,'base']=params[2]
    concentrations.loc[index,'base_sigma']=cv[2,2]
    full_data['fit'+row['Set']+str(row['Run'])]=monoExp(full_data['time'].values,params[0],params[1],params[2]) 


In [ ]:
concentrations['k']

In [ ]:
p0 = (1., 45., 0.05)
for index,row in concentrations.iterrows() : 
    x,y=np.split(full_data[full_data['time']>50][['time','abs'+row['Set']+str(row['Run'])]].dropna().values,2,1)
    params, cv = scipy.optimize.curve_fit(monoExp, x.flatten(), y.flatten(), p0)
    concentrations.loc[index,'scale']=params[0]
    concentrations.loc[index,'scale_sigma']=np.sqrt(cv[0,0])
    concentrations.loc[index,'k']=1./params[1]
    concentrations.loc[index,'k_sigma']=1./params[1]**2*np.sqrt(cv[1,1])
    concentrations.loc[index,'base']=params[2]
    concentrations.loc[index,'base_sigma']=cv[2,2]
    full_data['fit'+row['Set']+str(row['Run'])]=monoExp(full_data['time'].values,params[0],params[1],params[2]) 

In [ ]:
concentrations['k']

In [ ]:
#plt.semilogy(full_data['time'],full_data['absC3'])
#plt.semilogy(full_data['time'],full_data['fitC3'])
plt.semilogy(full_data['time'],full_data['absC3'],'o')
plt.semilogy(full_data['time'],full_data['fitC3'])


In [ ]:
# This cell is used to allow Google Colab to install the tools to convert the notebook to a pdf file
# Un-comment the following lines when you are ready to export the pdf 
#!apt-get install texlive texlive-xetex texlive-latex-extra pandoc
#!pip install pypandoc

In [ ]:
# Use this command to convert the finished worksheet into a pdf 
# NOTE : you may want to change the path of the file, if you are working in a different folder of the Google Drive
#!jupyter nbconvert --no-input --to PDF "/content/drive/MyDrive/Colab Notebooks/Kinetics_Worksheet.ipynb"